In [4]:
import pandas as pd
import numpy as np

# ^^^ pyforest auto-imports - don't write above this line


from sklearn.model_selection import TimeSeriesSplit
from scipy import stats
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

In [10]:
import pandas as pd
import numpy as np
# ^^^ pyforest auto-imports - don't write above this line
#FINALLY WORKING

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df["train_test"] = 'train'
df_test["train_test"] = "test"

print(df.shape)
print(df_test.shape)

df_combined = pd.concat([df,df_test])
print(df_combined.shape)

(3865, 81)
(890, 80)
(4755, 81)


/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [11]:
#Creating copy of combined dataset with reset index

df_combined_1 = df_combined.copy()
df_combined_1.reset_index(inplace=True,drop=False)
df_combined_1.drop("index", axis=1, inplace=True)
df_combined_1

,Adjusted net savings,"Adolescent birth rate (births per 1,000 female creatures ages 15-19)","Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Current health expenditure (% of GGP),Domestic credit provided by financial sector (% of GGP),Education Index,Employment in agriculture (% of total employment),Employment in services (% of total employment),Employment to population ratio (% ages 15 and older),"Estimated gross galactic income per capita, female",...,"Unemployment, youth (% ages 15–24)",Vulnerable employment (% of total employment),Young age (0-14) dependency ratio (per 100 creatures ages 15-64),Youth unemployment rate (female to male ratio),existence expectancy at birth,existence expectancy index,galactic year,galaxy,train_test,y
0,NaN,174.617899,NaN,NaN,75.604799,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,98.216072,NaN,63.125200,0.628657,990025,Large Magellanic Cloud (LMC),train,0.052590
1,19.177926,109.892385,NaN,NaN,57.214150,0.467873,NaN,NaN,NaN,NaN,...,NaN,NaN,88.322574,NaN,81.004994,0.818082,990025,Camelopardalis B,train,0.059868
2,21.151265,230.573666,6.534020,NaN,76.141735,0.363837,NaN,NaN,NaN,NaN,...,NaN,NaN,114.163995,NaN,59.570534,0.659443,990025,Virgo I,train,0.050449
3,NaN,198.971328,5.912194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,97.645232,NaN,52.333293,0.555862,990025,UGC 8651 (DDO 181),train,0.049394
4,NaN,61.008654,5.611753,NaN,NaN,0.918353,NaN,NaN,NaN,NaN,...,NaN,NaN,53.835875,NaN,81.802464,0.991196,990025,Tucana Dwarf,train,0.154247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1016064,ESO 383-087 (ISG 39),test,NaN
4751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1016064,NGC 247,test,NaN
4752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1016064,ESO 325-11,test,NaN
4753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1016064,ESO 540-032,test,NaN


In [12]:
#Fitting galactic year to date time format

df_combined['galactic year 1'] = df_combined['galactic year'].apply(lambda x:(np.round((x/1000),0))*2)
df_combined['galactic year 1'] = df_combined['galactic year 1'].astype('int64')
df_combined['galactic year 1'] = df_combined['galactic year 1'].astype('str')
df_combined['galactic year 1'] = df_combined['galactic year 1'] + "0101"
df_combined['date'] = pd.to_datetime(df_combined['galactic year 1'], yearfirst=True
                                 , format='%Y/%m/%d'
                                )
df_combined.head()

,Adjusted net savings,"Adolescent birth rate (births per 1,000 female creatures ages 15-19)","Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Current health expenditure (% of GGP),Domestic credit provided by financial sector (% of GGP),Education Index,Employment in agriculture (% of total employment),Employment in services (% of total employment),Employment to population ratio (% ages 15 and older),"Estimated gross galactic income per capita, female",...,Young age (0-14) dependency ratio (per 100 creatures ages 15-64),Youth unemployment rate (female to male ratio),existence expectancy at birth,existence expectancy index,galactic year,galaxy,train_test,y,galactic year 1,date
0,NaN,174.617899,NaN,NaN,75.604799,NaN,NaN,NaN,NaN,NaN,...,98.216072,NaN,63.125200,0.628657,990025,Large Magellanic Cloud (LMC),train,0.052590,19800101,1980-01-01
1,19.177926,109.892385,NaN,NaN,57.214150,0.467873,NaN,NaN,NaN,NaN,...,88.322574,NaN,81.004994,0.818082,990025,Camelopardalis B,train,0.059868,19800101,1980-01-01
2,21.151265,230.573666,6.534020,NaN,76.141735,0.363837,NaN,NaN,NaN,NaN,...,114.163995,NaN,59.570534,0.659443,990025,Virgo I,train,0.050449,19800101,1980-01-01
3,NaN,198.971328,5.912194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,97.645232,NaN,52.333293,0.555862,990025,UGC 8651 (DDO 181),train,0.049394,19800101,1980-01-01
4,NaN,61.008654,5.611753,NaN,NaN,0.918353,NaN,NaN,NaN,NaN,...,53.835875,NaN,81.802464,0.991196,990025,Tucana Dwarf,train,0.154247,19800101,1980-01-01


In [13]:
#ONLY USING FIRST 3 GALAXIES

mask = ((df_combined['galaxy']=='Large Magellanic Cloud (LMC)')|(df_combined['galaxy']=='Camelopardalis B')|(df_combined['galaxy']=='Virgo I'))
df_combined = df_combined[mask]
df_combined

,Adjusted net savings,"Adolescent birth rate (births per 1,000 female creatures ages 15-19)","Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Current health expenditure (% of GGP),Domestic credit provided by financial sector (% of GGP),Education Index,Employment in agriculture (% of total employment),Employment in services (% of total employment),Employment to population ratio (% ages 15 and older),"Estimated gross galactic income per capita, female",...,Young age (0-14) dependency ratio (per 100 creatures ages 15-64),Youth unemployment rate (female to male ratio),existence expectancy at birth,existence expectancy index,galactic year,galaxy,train_test,y,galactic year 1,date
0,NaN,174.617899,NaN,NaN,75.604799,NaN,NaN,NaN,NaN,NaN,...,98.216072,NaN,63.125200,0.628657,990025,Large Magellanic Cloud (LMC),train,0.052590,19800101,1980-01-01
1,19.177926,109.892385,NaN,NaN,57.214150,0.467873,NaN,NaN,NaN,NaN,...,88.322574,NaN,81.004994,0.818082,990025,Camelopardalis B,train,0.059868,19800101,1980-01-01
2,21.151265,230.573666,6.53402,NaN,76.141735,0.363837,NaN,NaN,NaN,NaN,...,114.163995,NaN,59.570534,0.659443,990025,Virgo I,train,0.050449,19800101,1980-01-01
194,NaN,NaN,NaN,NaN,NaN,NaN,96.827657,53.392606,76.506783,NaN,...,NaN,2.184798,62.389059,0.627245,991020,Large Magellanic Cloud (LMC),train,0.052115,19820101,1982-01-01
235,NaN,NaN,NaN,NaN,NaN,0.587530,68.266393,47.833230,62.562572,NaN,...,NaN,2.129355,73.687283,0.868862,991020,Camelopardalis B,train,0.059995,19820101,1982-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,43.022405,91.638031,NaN,5.952772,93.041138,0.577366,77.823432,47.088626,74.231191,18617.638573,...,56.111293,2.174437,83.695703,0.937656,1011030,Large Magellanic Cloud (LMC),test,NaN,20220101,2022-01-01
453,28.018268,84.011590,NaN,5.200907,124.595039,0.866781,46.532358,80.649626,64.376781,32871.255720,...,69.487423,1.679251,84.615662,0.951380,1016064,Camelopardalis B,test,NaN,20320101,2032-01-01
485,27.524615,80.321255,NaN,7.025313,138.305830,0.901802,50.268190,61.260724,58.336794,23903.316183,...,72.933001,2.348036,84.986949,0.964766,1015056,Camelopardalis B,test,NaN,20300101,2030-01-01
691,25.601998,88.551129,NaN,6.839673,155.682611,0.761121,35.195214,73.131443,53.878356,30331.703435,...,64.074680,2.532163,87.611550,1.034410,1012036,Camelopardalis B,test,NaN,20240101,2024-01-01


In [14]:
#Imputing nans with time series - trying on one galaxy

for g in df_combined.galaxy.unique():
    print(g)
    df_temp = df_combined[df_combined['galaxy']==g].copy()
    df = df_combined[~(df_combined['galaxy']==g)]
    #filling the Nans (first with interpolation and then backfilling if missing for start)
    df_temp.sort_values("galactic year",inplace=True)
    df_temp_1 = df_temp.copy()
    df_temp_1.drop(['galaxy','train_test','y','galactic year 1'
                    #'galactic year'
                   ],axis=1,inplace=True)
    df_temp_1.reset_index(inplace=True)
    null_list = [col for col in df_temp_1.columns if df_temp_1[col].isnull().all()]
    for n in null_list:
        df_temp_1.drop(columns=n, axis=1, inplace=True)
    
    print(null_list)
    
    for p in df_temp_1.columns:
        if p!='date':
            try:
                df_temp_temp_1 = df_temp_1[['date',p]].copy()
                df_temp_temp_1.set_index('date', inplace=True, drop=True)
                data = df_temp_temp_1
                #print(p)
                model = sm.tsa.statespace.SARIMAX(data, order=(1, 0, 0))
                #print("model initiated")
                results = model.fit(maxiter=5000)

                #option 1

                for q in df_temp_1.index:
                    if df_temp_1[p].loc[q]==np.nan:
                        df_temp_1[p].loc[q] = list(results.fittedvalues)[q]
                    else:
                        df_temp_1[p].loc[q]=df_temp_1[p].loc[q]
            except:
                #print(p)
                for q in df_temp_1.index:
                    if df_temp_1[p].loc[q]==np.nan:
                        df_temp_1[p].loc[q] = df_temp_1[p].mean()
                    else:
                        df_temp_1[p].loc[q]=df_temp_1[p].loc[q]
                    
                
    for n in null_list:
        df_temp_1[n]=list(df_temp[n])
        df_temp_1['galaxy'] = list(df_temp['galaxy'])
        df_temp_1['train_test'] = list(df_temp['train_test'])
        df_temp_1['y'] = list(df_temp['y'])


        results_df = pd.concat([results_df,df_temp_temp])
    
    df = pd.concat([df,df_temp_1])
    
df
#filling the Nans for any which didnt have anything to fill at all
#df.fillna(df.mean(),inplace=True)
    



Large Magellanic Cloud (LMC)
['Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total', 'Private galaxy capital flows (% of GGP)']


/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, Va

/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency informati

NameError: name 'results_df' is not defined

In [15]:
df

,Adjusted net savings,"Adolescent birth rate (births per 1,000 female creatures ages 15-19)","Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Current health expenditure (% of GGP),Domestic credit provided by financial sector (% of GGP),Education Index,Employment in agriculture (% of total employment),Employment in services (% of total employment),Employment to population ratio (% ages 15 and older),"Estimated gross galactic income per capita, female",...,Young age (0-14) dependency ratio (per 100 creatures ages 15-64),Youth unemployment rate (female to male ratio),existence expectancy at birth,existence expectancy index,galactic year,galaxy,train_test,y,galactic year 1,date
1,19.177926,109.892385,NaN,NaN,57.214150,0.467873,NaN,NaN,NaN,NaN,...,88.322574,NaN,81.004994,0.818082,990025,Camelopardalis B,train,0.059868,19800101,1980-01-01
2,21.151265,230.573666,6.534020,NaN,76.141735,0.363837,NaN,NaN,NaN,NaN,...,114.163995,NaN,59.570534,0.659443,990025,Virgo I,train,0.050449,19800101,1980-01-01
235,NaN,NaN,NaN,NaN,NaN,0.587530,68.266393,47.833230,62.562572,NaN,...,NaN,2.129355,73.687283,0.868862,991020,Camelopardalis B,train,0.059995,19820101,1982-01-01
309,NaN,NaN,NaN,NaN,NaN,0.342069,94.186618,35.848039,79.547832,NaN,...,NaN,2.301528,58.867426,0.555338,991020,Virgo I,train,0.049411,19820101,1982-01-01
364,NaN,NaN,NaN,NaN,NaN,0.390630,NaN,NaN,NaN,NaN,...,NaN,NaN,60.551216,0.554174,992016,Virgo I,train,0.049193,19840101,1984-01-01
477,NaN,NaN,NaN,NaN,NaN,0.602711,NaN,NaN,NaN,NaN,...,NaN,NaN,75.604580,0.788962,992016,Camelopardalis B,train,0.059934,19840101,1984-01-01
552,NaN,NaN,NaN,NaN,NaN,0.621111,NaN,NaN,NaN,NaN,...,NaN,NaN,80.028587,0.770042,993012,Camelopardalis B,train,0.059734,19860101,1986-01-01
635,NaN,NaN,NaN,NaN,NaN,0.348280,NaN,NaN,NaN,NaN,...,NaN,NaN,62.918772,0.612921,993012,Virgo I,train,0.048858,19860101,1986-01-01
769,NaN,NaN,NaN,NaN,NaN,0.564957,NaN,NaN,NaN,NaN,...,NaN,NaN,74.964164,0.875165,994009,Camelopardalis B,train,0.059479,19880101,1988-01-01
897,NaN,NaN,NaN,NaN,NaN,0.377290,NaN,NaN,NaN,NaN,...,NaN,NaN,50.942282,0.630974,994009,Virgo I,train,0.048516,19880101,1988-01-01
